In [1]:
# import libraries
import numpy as np
import sys
import psi4
from helper_PFCI import PFHamiltonianGenerator
np.set_printoptions(threshold=sys.maxsize)
psi4.core.set_output_file('output.dat', False)
import json
import git


In [2]:
# get commit number for github repository
repo = git.Repo(search_parent_directories=True)  
sha = repo.head.object.hexsha
print(sha)

319ea52b35e631a4b2a7ded8f56febf10b39eac6


In [3]:
# setup basic arguments for qed-ci calculation
mol_str = """
Li
H 1 1.8
symmetry c1
"""

options_dict = {
    "basis": "sto-3g",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
}

cavity_dict = {
    'omega_value' : 0.12086,
    'lambda_vector' : np.array([0, 0, 0.05]),
    'ci_level' : 'fci',   
    'full_diagonalization' : True,
    'number_of_photons' : 0, 
}


cavity_free_dict = {
    'omega_value' : 0.0,
    'lambda_vector' : np.array([0, 0, 0.0]),
    'ci_level' : 'fci',   
    'full_diagonalization' : True,
    'number_of_photons' : 0, 
}



In [4]:
# set up base dictionary - some of this will be updated with each calculation
dictionary = {
  "repo" : {
      
      "repository_url" : " https://github.com/FoleyLab/SCQED-PCQED",
      "branch" : "ruby_qed",
      "commit" : sha
  },
    
  "molecule" : {
    "molecule_name": "LiH",
    "geometry": [],
    "symbols": [
      "Li",
      "H"
    ],
    "z-matrix" : mol_str,
    "bond_length" : 1.8,
  },
    "driver": "energy",
    "model" : {
        "method" : "pCQED",
        "orbital_basis"  : options_dict["basis"],
        "photon_basis" : "number_basis",
        "number_of_photon_states" : 10,
        "number_of_electronic_states" : 2,
        
        "lambda" : [
            0,
            0,
            0,
        ],
        "omega" : 0,
    },
    
    "return_result" : [
        
    ],
    "cavity_free_rhf_energy" : 0.,
    "cavity_free_ci_ground_state_energy" : 0.,
    
}

# # function to generate file names based on system details
# def generate_file_name(dic):
# #     string_r = "{:.3f}".format(r)
#     file_name = dic["molecule"]["molecule_name"] + "_"
# #     file_name += "r_" + string_r + "_"
#     file_name += str(dictionary["model"]["method"]) + "_"
#     file_name += str(dictionary["model"]["orbital_basis"]) + "_"
#     file_name += str(dictionary["model"]["photon_basis"]) + "_"
#     file_name += str(dictionary["model"]["number_of_photon_states"]) + "_"
#     file_name += str(dictionary["model"]["number_of_electronic_states"]) + "_"
#     file_name += "lambda_" + str(dictionary["model"]["lambda"][0]) 
#     file_name += "_" + str(dictionary["model"]["lambda"][1]) 
#     file_name += "_" + str(dictionary["model"]["lambda"][2]) + "_"
#     file_name += "omega_" + str(dictionary["model"]["omega"]) + ".json"
#     return file_name

In [5]:
# print(dictionary)
# print(generate_file_name(dictionary))

In [6]:
mol_tmpl = """
Li
H 1 **R**
symmetry c1
"""

mol_str = """
Li
H 1 1.5
symmetry c1
"""

N_R = 50
N_el = 10
N_ph = 5
omega = 0.12086
lambda_vector = np.array([0., 0., 0.05])

r_array = np.linspace(1.4, 2.2, N_R)

# E_array = np.zeros((N_R, N_el))
en = []
r_idx = 0
for r in r_array:
#     file_name = generate_file_name(dictionary) 
    mol_str = mol_tmpl.replace("**R**", str(r))
    mol = psi4.geometry(mol_str)
    dictionary["molecule"]["z-matrix"] = mol_str
    dictionary["molecule"]["bond_length"] = r
    
    
    #psi4.set_options({'restricted_docc': [3],'active': [4],'num_roots':2})
    scf_e, wfn = psi4.energy('SCF/sto-3g', return_wfn=True)
    #fci_energy = psi4.energy('fci',ref_wfn=wfn)    
    LiH = PFHamiltonianGenerator(mol_str, options_dict, cavity_free_dict)
    #singlet_states = LiH.sort_dipole_allowed_states( 2)
    LiH.fast_build_pcqed_pf_hamiltonian(N_el, N_ph, omega, lambda_vector)
#     dictionary["model"]["number_of_photon_states"] = N_ph
#     dictionary["model"]["number_of_electronic_states"] = N_el
#     dictionary["model"]["lambda"] = [lambda_vector[0], lambda_vector[1], lambda_vector[2]]
#     dictionary["model"]["omega"] = omega
    
#     print(LiH.PCQED_H_PF)
#     if (N_el * N_ph) <=4:
#         dictionary["return_result"] = [LiH.PCQED_eigs[0], LiH.PCQED_eigs[1], LiH.PCQED_eigs[2], LiH.PCQED_eigs[3]]
        
#     else:
#         dictionary["return_result"] = [LiH.PCQED_eigs[0], LiH.PCQED_eigs[1], LiH.PCQED_eigs[2], LiH.PCQED_eigs[3], LiH.PCQED_eigs[4],LiH.PCQED_eigs[5]]
#     dictionary["cavity_free_rhf_energy"] = scf_e
#     dictionary["cavity_free_ci_ground_state_energy"] = LiH.CIeigs[0]
#     geometry = np.asarray(mol.geometry())
#     geometry_list = geometry.tolist()
#     dictionary["molecule"]["geometry"] = geometry_list
#     for i in range(N_el):
#         en.append(scf_e[r_idx, i])
#     dictionary["return_result"].append(en)
#     idx += 1





Start SCF iterations:

Canonical RHF One-electron energy = -12.6946346236296037
CQED-RHF One-electron energy      = -12.6946346236296037
Nuclear repulsion energy          = 1.1339511657214287
Dipole energy                     = 0.0000000000000000
SCF Iteration   1: Energy = -7.8605386610570473   dE = -7.86054E+00   dRMS = 1.78805E-13
SCF Iteration   2: Energy = -7.8605386610570385   dE =  8.88178E-15   dRMS = 3.97663E-14
Total time for SCF iterations: 0.000 seconds 

QED-RHF   energy: -7.86053866 hartree
Psi4  SCF energy: -7.86053866 hartree
 Completed QED-RHF in 0.08693194389343262 seconds
 Completed 1HSO Build in 5.817413330078125e-05 seconds
 Completed ERI Build in 0.0011289119720458984 seconds 
 Completed 2D build in 8.606910705566406e-05 seconds
 Completed 1G build in 1.4781951904296875e-05 seconds
 Completed the Dipole Matrix Build in 3.814697265625e-05 seconds
 Completed determinant list in 0.00044918060302734375 seconds 
 Completed constant offset matrix in 0.00014090538024902


Start SCF iterations:

Canonical RHF One-electron energy = -12.6723924436686470
CQED-RHF One-electron energy      = -12.6723924436686470
Nuclear repulsion energy          = 1.1208796825430838
Dipole energy                     = 0.0000000000000000
SCF Iteration   1: Energy = -7.8613487929934323   dE = -7.86135E+00   dRMS = 2.14738E-13
SCF Iteration   2: Energy = -7.8613487929934340   dE = -1.77636E-15   dRMS = 4.78008E-14
Total time for SCF iterations: 0.000 seconds 

QED-RHF   energy: -7.86134879 hartree
Psi4  SCF energy: -7.86134879 hartree
 Completed QED-RHF in 0.11138415336608887 seconds
 Completed 1HSO Build in 6.127357482910156e-05 seconds
 Completed ERI Build in 0.0012278556823730469 seconds 
 Completed 2D build in 0.00017189979553222656 seconds
 Completed 1G build in 1.811981201171875e-05 seconds
 Completed the Dipole Matrix Build in 4.410743713378906e-05 seconds
 Completed determinant list in 0.0004391670227050781 seconds 
 Completed constant offset matrix in 0.000164747238159


Start SCF iterations:

Canonical RHF One-electron energy = -12.6504263033321678
CQED-RHF One-electron energy      = -12.6504263033321678
Nuclear repulsion energy          = 1.1081061249072652
Dipole energy                     = 0.0000000000000000
SCF Iteration   1: Energy = -7.8620134490864411   dE = -7.86201E+00   dRMS = 1.88689E-13
SCF Iteration   2: Energy = -7.8620134490864411   dE =  0.00000E+00   dRMS = 4.20485E-14
Total time for SCF iterations: 0.000 seconds 

QED-RHF   energy: -7.86201345 hartree
Psi4  SCF energy: -7.86201345 hartree
 Completed QED-RHF in 0.09631109237670898 seconds
 Completed 1HSO Build in 6.198883056640625e-05 seconds
 Completed ERI Build in 0.0013911724090576172 seconds 
 Completed 2D build in 0.0001327991485595703 seconds
 Completed 1G build in 1.8835067749023438e-05 seconds
 Completed the Dipole Matrix Build in 4.124641418457031e-05 seconds
 Completed determinant list in 0.00045800209045410156 seconds 
 Completed constant offset matrix in 0.00017404556274


Start SCF iterations:

Canonical RHF One-electron energy = -12.6287313839641779
CQED-RHF One-electron energy      = -12.6287313839641779
Nuclear repulsion energy          = 1.0956204220914088
Dipole energy                     = 0.0000000000000000
SCF Iteration   1: Energy = -7.8625393734213223   dE = -7.86254E+00   dRMS = 2.34537E-13
SCF Iteration   2: Energy = -7.8625393734213187   dE =  3.55271E-15   dRMS = 5.24731E-14
Total time for SCF iterations: 0.000 seconds 

QED-RHF   energy: -7.86253937 hartree
Psi4  SCF energy: -7.86253937 hartree
 Completed QED-RHF in 0.08661198616027832 seconds
 Completed 1HSO Build in 5.602836608886719e-05 seconds
 Completed ERI Build in 0.0012359619140625 seconds 
 Completed 2D build in 8.726119995117188e-05 seconds
 Completed 1G build in 1.4781951904296875e-05 seconds
 Completed the Dipole Matrix Build in 3.910064697265625e-05 seconds
 Completed determinant list in 0.0004448890686035156 seconds 
 Completed constant offset matrix in 0.000141143798828125


Start SCF iterations:

Canonical RHF One-electron energy = -12.6073028869779620
CQED-RHF One-electron energy      = -12.6073028869779620
Nuclear repulsion energy          = 1.0834129522073814
Dipole energy                     = 0.0000000000000000
SCF Iteration   1: Energy = -7.8629329846331002   dE = -7.86293E+00   dRMS = 2.70251E-13
SCF Iteration   2: Energy = -7.8629329846330895   dE =  1.06581E-14   dRMS = 6.05218E-14
Total time for SCF iterations: 0.000 seconds 

QED-RHF   energy: -7.86293298 hartree
Psi4  SCF energy: -7.86293298 hartree
 Completed QED-RHF in 0.09557890892028809 seconds
 Completed 1HSO Build in 5.698204040527344e-05 seconds
 Completed ERI Build in 0.0012040138244628906 seconds 
 Completed 2D build in 0.00014281272888183594 seconds
 Completed 1G build in 1.621246337890625e-05 seconds
 Completed the Dipole Matrix Build in 4.100799560546875e-05 seconds
 Completed determinant list in 0.0004558563232421875 seconds 
 Completed constant offset matrix in 0.000177145004272

...


Start SCF iterations:

Canonical RHF One-electron energy = -11.9103500847605766
CQED-RHF One-electron energy      = -11.9103500847605766
Nuclear repulsion energy          = 0.7436811660467495
Dipole energy                     = 0.0000000000000000
SCF Iteration   1: Energy = -7.8157446684501251   dE = -7.81574E+00   dRMS = 3.25213E-13
SCF Iteration   2: Energy = -7.8157446684501037   dE =  2.13163E-14   dRMS = 8.73448E-14
Total time for SCF iterations: 0.000 seconds 

QED-RHF   energy: -7.81574467 hartree
Psi4  SCF energy: -7.81574467 hartree
 Completed QED-RHF in 0.09529280662536621 seconds
 Completed 1HSO Build in 5.125999450683594e-05 seconds
 Completed ERI Build in 0.0012879371643066406 seconds 
 Completed 2D build in 0.00011682510375976562 seconds
 Completed 1G build in 1.4066696166992188e-05 seconds
 Completed the Dipole Matrix Build in 3.695487976074219e-05 seconds
 Completed determinant list in 0.0004937648773193359 seconds 
 Completed constant offset matrix in 0.00018715858459


Start SCF iterations:

Canonical RHF One-electron energy = -11.8969117654643082
CQED-RHF One-electron energy      = -11.8969117654643082
Nuclear repulsion energy          = 0.7380365272152751
Dipole energy                     = 0.0000000000000000
SCF Iteration   1: Energy = -7.8138267017925394   dE = -7.81383E+00   dRMS = 1.07893E-12
SCF Iteration   2: Energy = -7.8138267017925394   dE =  0.00000E+00   dRMS = 2.92070E-13
Total time for SCF iterations: 0.000 seconds 

QED-RHF   energy: -7.81382670 hartree
Psi4  SCF energy: -7.81382670 hartree
 Completed QED-RHF in 0.0973818302154541 seconds
 Completed 1HSO Build in 7.486343383789062e-05 seconds
 Completed ERI Build in 0.0012769699096679688 seconds 
 Completed 2D build in 0.0001380443572998047 seconds
 Completed 1G build in 2.6941299438476562e-05 seconds
 Completed the Dipole Matrix Build in 4.9114227294921875e-05 seconds
 Completed determinant list in 0.0004801750183105469 seconds 
 Completed constant offset matrix in 0.000167846679687


Start SCF iterations:

Canonical RHF One-electron energy = -11.8836129583984018
CQED-RHF One-electron energy      = -11.8836129583984018
Nuclear repulsion energy          = 0.7324769300234460
Dipole energy                     = 0.0000000000000000
SCF Iteration   1: Energy = -7.8118950310418098   dE = -7.81190E+00   dRMS = 6.50003E-15
SCF Iteration   2: Energy = -7.8118950310418205   dE = -1.06581E-14   dRMS = 1.81652E-15
Total time for SCF iterations: 0.000 seconds 

QED-RHF   energy: -7.81189503 hartree
Psi4  SCF energy: -7.81189503 hartree
 Completed QED-RHF in 0.09069585800170898 seconds
 Completed 1HSO Build in 5.2928924560546875e-05 seconds
 Completed ERI Build in 0.0011730194091796875 seconds 
 Completed 2D build in 7.200241088867188e-05 seconds
 Completed 1G build in 1.4781951904296875e-05 seconds
 Completed the Dipole Matrix Build in 4.124641418457031e-05 seconds
 Completed determinant list in 0.0004439353942871094 seconds 
 Completed constant offset matrix in 0.00015616416931


Start SCF iterations:

Canonical RHF One-electron energy = -11.8704521229909634
CQED-RHF One-electron energy      = -11.8704521229909634
Nuclear repulsion energy          = 0.7270004669952337
Dipole energy                     = 0.0000000000000000
SCF Iteration   1: Energy = -7.8099506130038563   dE = -7.80995E+00   dRMS = 6.40319E-15
SCF Iteration   2: Energy = -7.8099506130038581   dE = -1.77636E-15   dRMS = 1.77232E-15
Total time for SCF iterations: 0.000 seconds 

QED-RHF   energy: -7.80995061 hartree
Psi4  SCF energy: -7.80995061 hartree
 Completed QED-RHF in 0.0937342643737793 seconds
 Completed 1HSO Build in 6.079673767089844e-05 seconds
 Completed ERI Build in 0.0015010833740234375 seconds 
 Completed 2D build in 0.0001430511474609375 seconds
 Completed 1G build in 1.6927719116210938e-05 seconds
 Completed the Dipole Matrix Build in 4.1961669921875e-05 seconds
 Completed determinant list in 0.00045800209045410156 seconds 
 Completed constant offset matrix in 0.00014591217041015


Start SCF iterations:

Canonical RHF One-electron energy = -11.8574277515713185
CQED-RHF One-electron energy      = -11.8574277515713185
Nuclear repulsion energy          = 0.7216052872772727
Dipole energy                     = 0.0000000000000000
SCF Iteration   1: Energy = -7.8079943691480960   dE = -7.80799E+00   dRMS = 6.11604E-15
SCF Iteration   2: Energy = -7.8079943691480942   dE =  1.77636E-15   dRMS = 1.68143E-15
Total time for SCF iterations: 0.000 seconds 

QED-RHF   energy: -7.80799437 hartree
Psi4  SCF energy: -7.80799437 hartree
 Completed QED-RHF in 0.09466314315795898 seconds
 Completed 1HSO Build in 5.602836608886719e-05 seconds
 Completed ERI Build in 0.0014710426330566406 seconds 
 Completed 2D build in 9.799003601074219e-05 seconds
 Completed 1G build in 1.6927719116210938e-05 seconds
 Completed the Dipole Matrix Build in 4.506111145019531e-05 seconds
 Completed determinant list in 0.0004508495330810547 seconds 
 Completed constant offset matrix in 0.000163078308105